#### Objective for IVSCC Manuscript Graphs

In [1]:
"""
Lockdown Date: 190830 for all graphs
nucleus_present vs nucleus_absent data:
-Restrict data collection range: anything with MET project code to 180620
-Because we only collect nucleus present after 180620
-MET data only

Nucleus_present analysis:
-present vs. absent

Comparion of present vs. absent with:
-cDNA quality (%>400 bp)
-Tree call(core, I1, I2, I3 and PoorQ)
-# of genes
-amplified content same as cDNA quantity (picogreen yield)
"""

'\nLockdown Date: 190830 for all graphs\nnucleus_present vs nucleus_absent data:\n-Restrict data collection range: anything with MET project code to 180620\n-Because we only collect nucleus present after 180620\n-MET data only\n\nNucleus_present analysis:\n-present vs. absent\n\nComparion of present vs. absent with:\n-cDNA quality (%>400 bp)\n-Tree call(core, I1, I2, I3 and PoorQ)\n-# of genes\n-amplified content same as cDNA quantity (picogreen yield)\n'

#### Imports

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [3]:
sns.set(context = "notebook", style = "ticks", font="verdana", font_scale = 1.35)
pd.set_option("display.max_colwidth",150) #Expands the number of characters shown in the columns
pd.set_option('display.max_columns', None)

#### Path variables

In [4]:
csv_path = "C:/Users/kumar/Documents/GitHub/Personal-Projects/ivscc_paper/csvs/"
#"C:/Users/ramr/Documents/Github/Personal-Projects/ivscc_paper/csvs/"
graph_path = "C:/Users/kumar/Documents/GitHub/Personal-Projects/ivscc_paper/graphs/"
#"C:/Users/ramr/Documents/Github/Personal-Projects/ivscc_paper/graphs/"

#### Misc

In [5]:
colors = ["#0039fa", "#fa0000"] #nucleus_present, nucleus_absent
#colors = ["#74c476", "#fb6a4a"] #nucleus_present, nucleus_absent
patch_order = ["nucleus_present", "nucleus_absent"]

#### Read Shiny mouse

In [6]:
shinym_df = pd.read_csv(csv_path + "shiny_mouse.csv")

IOError: File C:/Users/kumar/Documents/GitHub/Personal-Projects/ivscc_paper/csvs/shiny_mouse.csv does not exist

#### Filter to desired columns in shinym_df

In [ ]:
shinym_col_list = ["sample_id",
                   "Data",
                   "postPatch",
                   "percent_cdna_longer_than_400bp",
                   "amplified_quantity_ng",
                   "Genes.Detected.CPM",
                   "Tree_call",
                   "marker_sum_norm_label",
                   "Total_time"]
shinym_df = shinym_df[shinym_col_list]
len(shinym_df)

#### Filtering shinym_df based on conditions and creating/sorting by date column

In [ ]:
shinym_df.dropna(subset=["postPatch"], inplace=True) # 7 NaN samples
shinym_df["date"] = shinym_df.sample_id.str[5:11]
shinym_df.sort_values("date", inplace=True)
shinym_df.set_index("date", inplace=True)
shinym_df = shinym_df.loc[:"180620"]

In [ ]:
prod_df = shinym_df[shinym_df["Data"] == "Production"]
len(prod_df)

#### met_df has only cells with project code mIVSCC-MET

In [ ]:
met_df = prod_df[(prod_df.postPatch == "nucleus_present") | (prod_df.postPatch == "nucleus_absent")]
len(met_df)

#### Replacing all terminology to nucleus_present and nulceus_absent for all production data

In [ ]:
replacements = {"Nucleated" : "nucleus_present", 
                "Partial-Nucleus" : "nucleus_present",
                "No-Seal" : "nucleus_absent",
                "Entire-Cell" : "nucleus_absent",
                "entire_cell" : "nucleus_absent",
                "Outside-Out" : "nucleus_absent"}
prod_df["postPatch"] = prod_df["postPatch"].replace(replacements)

In [ ]:
prod_df.postPatch.value_counts()

In [ ]:
met_df.postPatch.value_counts()

In [ ]:
len(met_df)

#### Patch Duration Integration

In [ ]:
met_df.dropna(subset=["Total_time"], inplace=True)
len(met_df)

In [ ]:
met_df["Total_time_min"] = met_df["Total_time"] / 60

In [ ]:
bins = [0, 5, 10, 15, 20, 25]
labels = ["0-5", "5-10", "10-15", "15-20", "20-25"]
met_df = met_df[met_df["Total_time_min"] > 0] #& (met_df["Total_time_min"] <= 30)]
met_df["Total_time_bins"] = pd.cut(x=met_df["Total_time_min"], bins=bins, labels=labels)
len(met_df)

In [ ]:
met_df["percent_cdna_longer_than_400bp"] = met_df["percent_cdna_longer_than_400bp"] * 100

In [ ]:
met_df.head()

#### Obtaining mean markers sums for line plots

In [ ]:
met_amp = met_df.groupby(["Total_time_bins", "postPatch"])["amplified_quantity_ng"].agg(["mean", "sem", "std", "size"])
met_dna = met_df.groupby(["Total_time_bins", "postPatch"])["percent_cdna_longer_than_400bp"].agg(["mean", "sem", "std", "size"])
met_gen = met_df.groupby(["Total_time_bins", "postPatch"])["Genes.Detected.CPM"].agg(["mean", "sem", "std", "size"])
met_nms = met_df.groupby(["Total_time_bins", "postPatch"])["marker_sum_norm_label"].agg(["mean", "sem", "std", "size"])

met_amp.reset_index(inplace=True)
met_dna.reset_index(inplace=True)
met_gen.reset_index(inplace=True)
met_nms.reset_index(inplace=True)

In [ ]:
met_amp

In [ ]:
met_dna

In [ ]:
met_gen

In [ ]:
met_nms

#### Graph Function for Line Graph

In [ ]:
def linplt(col_label, df):
    ax=sns.catplot(kind="point", x="Total_time_bins", y="mean", hue="postPatch", hue_order=patch_order, data=df,
                   palette=colors, saturation=1, width=0.6, aspect=1.3, legend=False)
    ax.set(xlabel="Patch Duration (min)", ylabel=col_label)

In [ ]:
def poiplt(col, col_label, df, ymax):
    plt.figure(figsize=(6, 6))
    ax=sns.pointplot(x="Total_time_bins", y=col, hue="postPatch", hue_order=patch_order,
                     data=df, palette=colors, dodge=True,
                     markers=["o", "o"], ci=68, scale=1.5, capsize=0.2, errwidth=1.5)
    ax.legend_.remove()
    ax.set(xlabel="Patch Duration (min)", ylabel=col_label)
    ax.set_title(col_label)
    ax.set_ylim(0, ymax)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.savefig(graph_path + "met_poi" + col + ".jpeg", bbox_inches="tight")

#### Graph Function for Boxen Graph

In [ ]:
def benplt(col, col_label, df):
    ax=sns.catplot(kind="boxen", x="Total_time_bins", y=col, hue="postPatch", 
                   hue_order=["nucleus_present", "nucleus_absent"], 
                   data=df, palette=colors, saturation=1, width=0.6, aspect=2, legend=False)
    ax.set(xlabel="Patch Duration (min)", ylabel=col_label)
    #plt.legend(frameon=True, fancybox=True, title="Post Patch Outcome",
    #           loc="center left", bbox_to_anchor=(1.0, 0.5), shadow=True)
    plt.savefig(graph_path + "met_ben" + col + ".jpeg", bbox_inches="tight")

#### Graph Function for Violin Graph

In [ ]:
def vioplt(col, col_label, df):
    plt.figure(figsize=(5, 6))
    ax=sns.violinplot(x="postPatch", y=col, data=df, order=patch_order, 
                      scale="count", inner="quartile", 
                      linewidth=1, saturation=1, width=0.9, palette=colors)
    ax=sns.swarmplot(x="postPatch", y=col, data=df, order=patch_order,
                     linewidth=1, size=0.3, color="0.2")
    ax.set_xlabel("Post Patch Outcome")
    ax.set_ylabel(col_label, fontsize=16)
    plt.savefig(graph_path + "met_vio" + col + ".jpeg", bbox_inches="tight")

#### Graph Functions for Stacked Percentage Bar Graph

In [ ]:
def piv_table(df):
    df1 = df.loc[:,["postPatch", "Tree_call", "Data"]]
    df1 = df.groupby(["postPatch", "Tree_call"]).count()
    df1 = df1.rename(columns = {"Data" : ""})
    df1.reset_index(inplace = True)
    df1 = df1.pivot_table(values=[''], index=["postPatch"], columns = ["Tree_call"], aggfunc="sum")
    return df1

def stacked_plot(df):
    nuc_order = ["nucleus_present", "nucleus_absent"]
    my_colors = ['#74c476', '#de2d26']
    bp = df.loc[nuc_order].plot(kind="bar", stacked = True, figsize= (8,6), rot = 0, 
                 colormap=ListedColormap(sns.color_palette("GnBu", 10)),width = 0.4)

    bp.legend(["Core", "I1", "I2", "I3", "PoorQ"], loc = 0, bbox_to_anchor = (1, 1.02), 
              frameon = True, shadow = True, fontsize = 13)

    plt.subplots_adjust(left = 0.1, right = 0.8, bottom = None, top = None, wspace=None, hspace=None)
    #This helps if axis labels are getting cutoff when saving final image

    #Rusty's Method figure it out later
    rects = bp.patches
    labels = post_patch_totals

    #Rusty's Method figure it out later
    for rect, label in zip (rects, labels):
        height = 100
        x_value = rect.get_x() + rect.get_width() / 2
        bp.text(rect.get_x() + rect.get_width()/2, height, label, ha='center', va='bottom', size = 12)

    bp.set_title("Tree calls for Post Patch Outcomes")
    bp.set(xlabel = "Post Patch Outcome", ylabel = "Percentage")
    plt.savefig(graph_path + "met_stk.jpeg", bbox_inches="tight")

#### Line Graphs

In [ ]:
met_df.postPatch.value_counts()

In [ ]:
poiplt("amplified_quantity_ng", "Amplified content (ng)", met_df, 20)
poiplt("percent_cdna_longer_than_400bp", "cDNA quality (%>400 bp)", met_df, 100)
poiplt("Genes.Detected.CPM", "Number of genes", met_df, 10000)
poiplt("marker_sum_norm_label", "NMS", met_df, 1.0)

#### Boxen Graphs

In [ ]:
benplt("amplified_quantity_ng", "Amplified content (ng)", met_df[met_df.amplified_quantity_ng <= 60])
benplt("percent_cdna_longer_than_400bp", "cDNA quality (%>400 bp)", met_df[met_df.percent_cdna_longer_than_400bp <= 1])
benplt("Genes.Detected.CPM", "Number of genes", met_df[met_df["Genes.Detected.CPM"] <= 15000])
benplt("marker_sum_norm_label", "NMS", met_df[met_df.marker_sum_norm_label <= 1.5])

#### Violin Graphs

In [ ]:
vioplt("amplified_quantity_ng", "Amplified content (ng)", met_df[met_df.amplified_quantity_ng <= 60])
vioplt("percent_cdna_longer_than_400bp", "cDNA quality (%>400 bp)", met_df[met_df.percent_cdna_longer_than_400bp <= 1])
vioplt("Genes.Detected.CPM", "Number of genes", met_df[met_df["Genes.Detected.CPM"] <= 15000])
vioplt("marker_sum_norm_label", "NMS", met_df[met_df.marker_sum_norm_label <= 1.5])

#### Stacked Percentage Bar Graph

In [ ]:
met_df1 = piv_table(met_df)
post_patch_totals = list(met_df1.sum(1))
post_patch_totals = [int(x) for x in post_patch_totals]
met_df1 = met_df1.div(met_df1.sum(1), axis=0) * 100
met_df1

In [ ]:
stacked_plot(met_df1)

#### Countplot for Tree_calls for nucelus_present vs nucleus_absent

In [ ]:
sns.countplot(x="postPatch", hue="Tree_call", hue_order = ["Core", "I1", "I2", "I3", "PoorQ"],
              data=met_df, palette="GnBu", saturation=1)

#### Writing dataframe into Excel Sheets

In [ ]:
writer = pd.ExcelWriter(graph_path + "nucleus_present_absent_RR.xlsx")
met_df.to_excel(writer, "MET Data", freeze_panes=(1,0))
prod_df.to_excel(writer, "Production Data", freeze_panes=(1,0))
shinym_df.to_excel(writer, "Reference Shiny Mouse Data", freeze_panes=(1,0))
writer.save()